# Week 12: Working With Continuous Data

Source: [MNE-Python](https://mne.tools/stable/index.html)

## 1. The Raw data structure: continuous data

This tutorial covers the basics of working with raw EEG/MEG data in Python. It introduces the `~mne.io.Raw` data structure in detail, including how to load, query, subselect, export, and plot data from a `~mne.io.Raw` object. For more info on visualization of `~mne.io.Raw` objects, see `tut-visualize-raw`. For info on creating a `~mne.io.Raw` object from simulated data in a `NumPy array <numpy.ndarray>`, see `tut-creating-data-structures`.

As usual we'll start by importing the modules we need:

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np

import mne

### Loading continuous data

Datasets in MNE-Python:
```
There are data_path functions for several example datasets in MNE-Python (e.g., mne.datasets.kiloword.data_path(), mne.datasets.spm_face.data_path(), etc). All of them will check the default download location first to see if the dataset is already on your computer, and only download it if necessary. The default download location is also configurable; see the documentation of any of the data_path functions for more information.
```

As mentioned in `the introductory tutorial <tut-overview>`, MNE-Python data structures are based around the `.fif` file format from Neuromag. This tutorial uses an `example dataset <sample-dataset>` in `.fif` format, so here we'll use the function `mne.io.read_raw_fif` to load the raw data; there are reader functions for `a wide variety of other data formats <data-formats>` as well.

There are also `several other example datasets <datasets>` that can be downloaded with just a few lines of code. Functions for downloading example datasets are in the `mne.datasets` submodule; here we'll use `mne.datasets.sample.data_path` to download the "`sample-dataset`" dataset, which contains EEG, MEG, and structural MRI data from one subject performing an audiovisual experiment. When it's done downloading, `~mne.datasets.sample.data_path` will return the folder location where it put the files; you can navigate there with your file browser if you want to examine the files yourself. Once we have the file path, we can load the data with `~mne.io.read_raw_fif`. This will return a `~mne.io.Raw` object, which we'll store in a variable called ``raw``.

In [ ]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(
    sample_data_folder, "MEG", "sample", "sample_audvis_raw.fif"
)
raw = mne.io.read_raw_fif(sample_data_raw_file)

As you can see above, `~mne.io.read_raw_fif` automatically displays some information about the file it's loading. For example, here it tells us that there are three "projection items" in the file along with the recorded
data; those are `SSP projectors <projector>` calculated to remove environmental noise from the MEG signals, and are discussed in a the tutorial `tut-projectors-background`. In addition to the information displayed during loading, you can get a glimpse of the basic details of a `~mne.io.Raw` object by printing it:

In [ ]:
print(raw)

By default, the `mne.io.read_raw_{*}` family of functions will *not* load the data into memory (instead the data on disk are `memory-mapped`_, meaning the data are only read from disk as-needed). Some operations (such as
filtering) require that the data be copied into RAM; to do that we could have passed the ``preload=True`` parameter to `~mne.io.read_raw_fif`, but we can also copy the data into RAM at any time using the
`~mne.io.Raw.load_data` method. However, since this particular tutorial doesn't do any serious analysis of the data, we'll first `~mne.io.Raw.crop` the `~mne.io.Raw` object to 60 seconds so it uses less memory and runs more smoothly on our documentation server.

In [ ]:
raw.crop(tmax=60)

### Querying the Raw object

Attributes vs. Methods:

```
Attributes are usually static properties of Python objects — things that are pre-computed and stored as part of the object’s representation in memory. Attributes are accessed with the . operator and do not require parentheses after the attribute name (example: raw.ch_names).

Methods are like specialized functions attached to an object. Usually they require additional user input and/or need some computation to yield a result. Methods always have parentheses at the end; additional arguments (if any) go inside those parentheses (examples: raw.estimate_rank(), raw.drop_channels(['EEG 030', 'MEG 2242'])).
```

We saw above that printing the `~mne.io.Raw` object displays some basic information like the total number of channels, the number of time points at which the data were sampled, total duration, and the approximate size in memory. Much more information is available through the various *attributes* and *methods* of the `~mne.io.Raw` class. Some useful attributes of `~mne.io.Raw` objects include a list of the channel names (`~mne.io.Raw.ch_names`), an array of the sample times in seconds (`~mne.io.Raw.times`), and the total number of samples (`~mne.io.Raw.n_times`); a list of all attributes and methods is given in the documentation of the `~mne.io.Raw` class.

### The ``Raw.info`` attribute

There is also quite a lot of information stored in the ``raw.info`` attribute, which stores an `~mne.Info` object that is similar to a `Python dictionary <dict>` (in that it has fields accessed via named keys). Like Python dictionaries, ``raw.info`` has a ``.keys()`` method that shows all the available field names; unlike Python dictionaries, printing ``raw.info`` will print a nicely-formatted glimpse of each field's data. See `tut-info-class` for more on what is stored in `~mne.Info` objects, and how to interact with them.

In [ ]:
n_time_samps = raw.n_times
time_secs = raw.times
ch_names = raw.ch_names
n_chan = len(ch_names)  # note: there is no raw.n_channels attribute
print(
    f"the (cropped) sample data object has {n_time_samps} time samples and "
    f"{n_chan} channels."
)
print(f"The last time sample is at {time_secs[-1]} seconds.")
print("The first few channel names are {}.".format(", ".join(ch_names[:3])))
print()  # insert a blank line in the output

# some examples of raw.info:
print("bad channels:", raw.info["bads"])  # chs marked "bad" during acquisition
print(raw.info["sfreq"], "Hz")  # sampling frequency
print(raw.info["description"], "\n")  # miscellaneous acquisition info

print(raw.info)

**Note:**

```
Most of the fields of raw.info reflect metadata recorded at acquisition time, and should not be changed by the user. There are a few exceptions (such as raw.info['bads'] and raw.info['projs']), but in most cases there are dedicated MNE-Python functions or methods to update the Info object safely (such as add_proj() to update raw.info['projs']).
```

### Time, sample number, and sample index

Sample numbering in VectorView data:

```
For data from VectorView systems, it is important to distinguish sample number from sample index. See first_samp for more information.
```

One method of `Raw` objects that is frequently useful is `time_as_index()`, which converts a time (in seconds) into the integer index of the sample occurring closest to that time. The method can also take a list or array of times, and will return an array of indices.

It is important to remember that there may not be a data sample at exactly the time requested, so the number of samples between `time = 1` second and `time = 2` seconds may be different than the number of samples between `time = 2` and `time = 3`:

In [ ]:
print(raw.time_as_index(20))
print(raw.time_as_index([20, 30, 40]), "\n")

print(np.diff(raw.time_as_index([1, 2, 3])))

### Modifying ``Raw`` objects

**len(raw):**

```
Although the Raw object underlyingly stores data samples in a NumPy array of shape (n_channels, n_timepoints), the Raw object behaves differently from NumPy arrays with respect to the len() function. len(raw) will return the number of timepoints (length along data axis 1), not the number of channels (length along data axis 0). Hence in this section you’ll see len(raw.ch_names) to get the number of channels.
```

`~mne.io.Raw` objects have a number of methods that modify the `~mne.io.Raw` instance in-place and return a reference to the modified instance. This can be useful for `method chaining`_
(e.g., ``raw.crop(...).pick(...).filter(...).plot()``) but it also poses a problem during interactive analysis: if you modify your `~mne.io.Raw` object for an exploratory plot or analysis (say, by dropping some channels), you will then need to re-load the data (and repeat any earlier processing steps) to undo the channel-dropping and try something else. For that reason, the examples in this section frequently use the `~mne.io.Raw.copy` method before the other methods being demonstrated, so that the original `~mne.io.Raw` object is still available in the variable ``raw`` for use in later examples.

### Selecting, dropping, and reordering channels

Altering the channels of a `~mne.io.Raw` object can be done in several ways. As a first example, we'll use the `~mne.io.Raw.pick` method to restrict the `~mne.io.Raw` object to just the EEG and EOG channels:

In [ ]:
eeg_and_eog = raw.copy().pick(picks=["eeg", "eog"])
print(len(raw.ch_names), "→", len(eeg_and_eog.ch_names))

In addition, `~mne.io.Raw.pick` can also be used to pick channels by name, and a
corresponding `~mne.io.Raw.drop_channels` method to remove channels by
name:

In [ ]:
raw_temp = raw.copy()
print("Number of channels in raw_temp:")
print(len(raw_temp.ch_names), end=" → drop two → ")
raw_temp.drop_channels(["EEG 037", "EEG 059"])
print(len(raw_temp.ch_names), end=" → pick three → ")
raw_temp.pick(["MEG 1811", "EEG 017", "EOG 061"])
print(len(raw_temp.ch_names))

If you want the channels in a specific order (e.g., for plotting), `~mne.io.Raw.reorder_channels` works just like `~mne.io.Raw.pick` but also reorders the channels; for example, here we pick the EOG and frontal EEG channels, putting the EOG first and the EEG in reverse order:

In [ ]:
channel_names = ["EOG 061", "EEG 003", "EEG 002", "EEG 001"]
eog_and_frontal_eeg = raw.copy().reorder_channels(channel_names)
print(eog_and_frontal_eeg.ch_names)

### Changing channel name and type

**Long channel names:**

```
Due to limitations in the .fif file format (which MNE-Python uses to save Raw objects), channel names are limited to a maximum of 15 characters.
```

You may have noticed that the EEG channel names in the sample data are numbered rather than labelled according to a standard nomenclature such as the `10-20` or `10-05` systems, or perhaps it bothers you that the channel names contain spaces. It is possible to rename channels using the `rename_channels()` method, which takes a Python dictionary to map old names to new names. You need not rename all channels at once; provide only the dictionary entries for the channels you want to rename. Here’s a frivolous example:

In [ ]:
raw.rename_channels({"EOG 061": "blink detector"})

This next example replaces spaces in the channel names with underscores, using a Python `dict comprehension`_:

In [ ]:
print(raw.ch_names[-3:])
channel_renaming_dict = {name: name.replace(" ", "_") for name in raw.ch_names}
raw.rename_channels(channel_renaming_dict)
print(raw.ch_names[-3:])

If for some reason the channel types in your `Raw` object are inaccurate, you can change the type of any channel with the `set_channel_types()` method. The method takes a `dictionary` mapping channel names to types; allowed types are `bio, chpi, csd, dbs, dipole, ecg, ecog, eeg, emg, eog, exci, eyegaze, fnirs_cw_amplitude, fnirs_fd_ac_amplitude, fnirs_fd_phase, fnirs_od, gof, gsr, hbo, hbr, ias, misc, pupil, ref_meg, resp, seeg, stim, syst, temperature` (see sensor types for more information about them). A common use case for changing channel type is when using frontal EEG electrodes as makeshift EOG channels:

In [ ]:
raw.set_channel_types({"EEG_001": "eog"})
print(raw.copy().pick(picks="eog").ch_names)

### Selection in the time domain

If you want to limit the time domain of a `~mne.io.Raw` object, you can use the `~mne.io.Raw.crop` method, which modifies the `~mne.io.Raw` object in place (we've seen this already at the start of this tutorial, when we cropped the `~mne.io.Raw` object to 60 seconds to reduce memory demands). `~mne.io.Raw.crop` takes parameters ``tmin`` and ``tmax``, both in seconds (here we'll again use `~mne.io.Raw.copy` first to avoid changing the original `~mne.io.Raw` object):

In [ ]:
raw_selection = raw.copy().crop(tmin=10, tmax=12.5)
print(raw_selection)

`~mne.io.Raw.crop` also modifies the `~mne.io.Raw.first_samp` and
`~mne.io.Raw.times` attributes, so that the first sample of the cropped
object now corresponds to ``time = 0``. Accordingly, if you wanted to re-crop
``raw_selection`` from 11 to 12.5 seconds (instead of 10 to 12.5 as above)
then the subsequent call to `~mne.io.Raw.crop` should get ``tmin=1``
(not ``tmin=11``), and leave ``tmax`` unspecified to keep everything from
``tmin`` up to the end of the object:

In [ ]:
print(raw_selection.times.min(), raw_selection.times.max())
raw_selection.crop(tmin=1)
print(raw_selection.times.min(), raw_selection.times.max())

Remember that sample times don't always align exactly with requested ``tmin``
or ``tmax`` values (due to sampling), which is why the ``max`` values of the
cropped files don't exactly match the requested ``tmax`` (see
`time-as-index` for further details).

If you need to select discontinuous spans of a `~mne.io.Raw` object —
or combine two or more separate `~mne.io.Raw` objects — you can use
the `~mne.io.Raw.append` method:

In [ ]:
raw_selection1 = raw.copy().crop(tmin=30, tmax=30.1)  # 0.1 seconds
raw_selection2 = raw.copy().crop(tmin=40, tmax=41.1)  # 1.1 seconds
raw_selection3 = raw.copy().crop(tmin=50, tmax=51.3)  # 1.3 seconds
raw_selection1.append([raw_selection2, raw_selection3])  # 2.5 seconds total
print(raw_selection1.times.min(), raw_selection1.times.max())

<div class="alert alert-danger"><h4>Warning</h4><p>Be careful when concatenating :class:`~mne.io.Raw` objects from different
    recordings, especially when saving: `~mne.io.Raw.append` only
    preserves the ``info`` attribute of the initial `~mne.io.Raw`
    object (the one outside the `~mne.io.Raw.append` method call).</p></div>
    
    
### Extracting data from ``Raw`` objects

So far we've been looking at ways to modify a `~mne.io.Raw` object.
This section shows how to extract the data from a `~mne.io.Raw` object
into a `NumPy array <numpy.ndarray>`, for analysis or plotting using
functions outside of MNE-Python. To select portions of the data,
`~mne.io.Raw` objects can be indexed using square brackets. However,
indexing `~mne.io.Raw` works differently than indexing a `NumPy
array <numpy.ndarray>` in two ways:

1. Along with the requested sample value(s) MNE-Python also returns an array
   of times (in seconds) corresponding to the requested samples. The data
   array and the times array are returned together as elements of a tuple.

2. The data array will always be 2-dimensional even if you request only a
   single time sample or a single channel.


### Extracting data by index

To illustrate the above two points, let's select a couple seconds of data
from the first channel:

In [ ]:
sampling_freq = raw.info["sfreq"]
start_stop_seconds = np.array([11, 13])
start_sample, stop_sample = (start_stop_seconds * sampling_freq).astype(int)
channel_index = 0
raw_selection = raw[channel_index, start_sample:stop_sample]
print(raw_selection)

You can see that it contains 2 arrays. This combination of data and times makes it easy to plot selections of raw data (although note that we’re transposing the data array so that each channel is a column instead of a row, to match what matplotlib expects when plotting 2-dimensional `y` against 1-dimensional `x`):

In [ ]:
x = raw_selection[1]
y = raw_selection[0].T
plt.plot(x, y)

### Extracting channels by name

The `~mne.io.Raw` object can also be indexed with the names of
channels instead of their index numbers. You can pass a single string to get
just one channel, or a list of strings to select multiple channels. As with
integer indexing, this will return a tuple of ``(data_array, times_array)``
that can be easily plotted. Since we're plotting 2 channels this time, we'll
add a vertical offset to one channel so it's not plotted right on top
of the other one:

In [ ]:
channel_names = ["MEG_0712", "MEG_1022"]
two_meg_chans = raw[channel_names, start_sample:stop_sample]
y_offset = np.array([5e-11, 0])  # just enough to separate the channel traces
x = two_meg_chans[1]
y = two_meg_chans[0].T + y_offset
lines = plt.plot(x, y)
plt.legend(lines, channel_names)

### Extracting channels by type

There are several ways to select all channels of a given type from a
`~mne.io.Raw` object. The safest method is to use
`mne.pick_types` to obtain the integer indices of the channels you
want, then use those indices with the square-bracket indexing method shown
above. The `~mne.pick_types` function uses the `~mne.Info`
attribute of the `~mne.io.Raw` object to determine channel types, and
takes boolean or string parameters to indicate which type(s) to retain. The
``meg`` parameter defaults to ``True``, and all others default to ``False``,
so to get just the EEG channels, we pass ``eeg=True`` and ``meg=False``:

In [ ]:
eeg_channel_indices = mne.pick_types(raw.info, meg=False, eeg=True)
eeg_data, times = raw[eeg_channel_indices]
print(eeg_data.shape)

Some of the parameters of `mne.pick_types` accept string arguments as
well as booleans. For example, the ``meg`` parameter can take values
``'mag'``, ``'grad'``, ``'planar1'``, or ``'planar2'`` to select only
magnetometers, all gradiometers, or a specific type of gradiometer. See the
docstring of `mne.pick_types` for full details.


### The ``Raw.get_data()`` method

If you only want the data (not the corresponding array of times),
`~mne.io.Raw` objects have a `~mne.io.Raw.get_data` method. Used
with no parameters specified, it will extract all data from all channels, in
a (n_channels, n_timepoints) `NumPy array <numpy.ndarray>`:

In [ ]:
data = raw.get_data()
print(data.shape)

If you want the array of times, `~mne.io.Raw.get_data` has an optional ``return_times`` parameter:

In [ ]:
data, times = raw.get_data(return_times=True)
print(data.shape)
print(times.shape)

The `~mne.io.Raw.get_data` method can also be used to extract specific
channel(s) and sample ranges, via its ``picks``, ``start``, and ``stop``
parameters. The ``picks`` parameter accepts integer channel indices, channel
names, or channel types, and preserves the requested channel order given as
its ``picks`` parameter.

In [ ]:
first_channel_data = raw.get_data(picks=0)
eeg_and_eog_data = raw.get_data(picks=["eeg", "eog"])
two_meg_chans_data = raw.get_data(picks=["MEG_0712", "MEG_1022"], start=1000, stop=2000)

print(first_channel_data.shape)
print(eeg_and_eog_data.shape)
print(two_meg_chans_data.shape)

### Summary of ways to extract data from ``Raw`` objects

The following table summarizes the various ways of extracting data from a `Raw` object.


######### INSERT TABLE HERE ############

########################################

########################################

## Exporting and saving Raw objects

`~mne.io.Raw` objects have a built-in `~mne.io.Raw.save` method,
which can be used to write a partially processed `~mne.io.Raw` object
to disk as a `.fif` file, such that it can be re-loaded later with its
various attributes intact (but see `precision` for an important
note about numerical precision when saving).

There are a few other ways to export just the sensor data from a
`~mne.io.Raw` object. One is to use indexing or the
`~mne.io.Raw.get_data` method to extract the data, and use
`numpy.save` to save the data array:

In [ ]:
data = raw.get_data()
np.save(file="my_data.npy", arr=data)

It is also possible to export the data to a `Pandas DataFrame
<pandas.DataFrame>` object, and use the saving methods that `Pandas
<pandas>` affords. The `~mne.io.Raw` object's
`~mne.io.Raw.to_data_frame` method is similar to
`~mne.io.Raw.get_data` in that it has a ``picks`` parameter for
restricting which channels are exported, and ``start`` and ``stop``
parameters for restricting the time domain. Note that, by default, times will
be converted to milliseconds, rounded to the nearest millisecond, and used as
the DataFrame index; see the ``scaling_time`` parameter in the documentation
of `~mne.io.Raw.to_data_frame` for more details.

In [ ]:
sampling_freq = raw.info["sfreq"]
start_end_secs = np.array([10, 13])
start_sample, stop_sample = (start_end_secs * sampling_freq).astype(int)
df = raw.to_data_frame(picks=["eeg"], start=start_sample, stop=stop_sample)
# then save using df.to_csv(...), df.to_hdf(...), etc
print(df.head())

<div class="alert alert-info"><h4>Note</h4><p>When exporting data as a `NumPy array <numpy.ndarray>` or
    `Pandas DataFrame <pandas.DataFrame>`, be sure to properly account
    for the `unit of representation <units>` in your subsequent
    analyses.</p></div>

## 2. Working with events


This tutorial describes event representation and how event arrays are used to
subselect data.

As usual we'll start by importing the modules we need, loading some
`example data <sample-dataset>`, and cropping the `~mne.io.Raw`
object to just 60 seconds before loading it into RAM to save memory:

In [ ]:
import os

import numpy as np

import mne

sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(
    sample_data_folder, "MEG", "sample", "sample_audvis_raw.fif"
)
raw = mne.io.read_raw_fif(sample_data_raw_file, verbose=False)
raw.crop(tmax=60).load_data()

The tutorial `tut-events-vs-annotations` describes in detail the
different ways of obtaining an `Events array <events>` from a
`~mne.io.Raw` object (see the section
`overview-tut-events-section` for details). Since the `sample
dataset <sample-dataset>` includes experimental events recorded on
`stim channel` ``STI 014``, we'll start this tutorial by parsing the
events from that channel using `mne.find_events`:

In [ ]:
events = mne.find_events(raw, stim_channel="STI 014")

### Reading and writing events from/to a file

Event arrays are `NumPy array <numpy.ndarray>` objects, so they could
be saved to disk as binary `.npy` files using `numpy.save`.
However, MNE-Python provides convenience functions `mne.read_events`
and `mne.write_events` for reading and writing event arrays as either
text files (common file extensions are `.eve`, `.lst`, and
`.txt`) or binary `.fif` files. The example dataset includes the
results of ``mne.find_events(raw)`` in a `.fif` file. Since we've
truncated our `~mne.io.Raw` object, it will have fewer events than the
events file loaded from disk (which contains events for the entire
recording), but the events should match for the first 60 seconds anyway:

In [ ]:
sample_data_events_file = os.path.join(
    sample_data_folder, "MEG", "sample", "sample_audvis_raw-eve.fif"
)
events_from_file = mne.read_events(sample_data_events_file)
assert np.array_equal(events, events_from_file[: len(events)])

When writing event arrays to disk, the format will be inferred from the file
extension you provide. By convention, MNE-Python expects events files to
either have an `.eve` extension or to have a file basename ending in
``-eve`` or ``_eve`` (e.g., `{my_experiment}_eve.fif`), and will issue
a warning if this convention is not respected.


### Subselecting and combining events

The output of `~mne.find_events` above (repeated here) told us the
number of events that were found, and the unique integer event IDs present:

In [ ]:
mne.find_events(raw, stim_channel="STI 014")

**Including/excluding events:**

```
Just like pick_events, read_events also has include and exclude parameters.
```

If some of those events are not of interest, you can easily subselect events
using `mne.pick_events`, which has parameters ``include`` and
``exclude``. For example, in the sample data Event ID 32 corresponds to a
subject button press, which could be excluded as:

In [ ]:
events_no_button = mne.pick_events(events, exclude=32)

It is also possible to combine two Event IDs using `mne.merge_events`; the following example will combine Event IDs 1, 2 and 3 into a single event labelled ``1``:

In [ ]:
merged_events = mne.merge_events(events, [1, 2, 3], 1)
print(np.unique(merged_events[:, -1]))

Note, however, that merging events is not necessary if you simply want to
pool trial types for analysis; the next section describes how MNE-Python uses
*event dictionaries* to map integer Event IDs to more descriptive label
strings.


### Mapping Event IDs to trial descriptors

So far in this tutorial we've only been dealing with integer Event IDs, which
were assigned based on DC voltage pulse magnitude (which is ultimately
determined by the experimenter's choices about what signals to send to the
STIM channels). Keeping track of which Event ID corresponds to which
experimental condition can be cumbersome, and it is often desirable to pool
experimental conditions during analysis. You may recall that the mapping of
integer Event IDs to meaningful descriptions for the `sample dataset
<sample-dataset>` is given in `this table
<sample-data-event-dict-table>` in the `introductory tutorial
<tut-overview>`. Here we simply reproduce that mapping as an
*event dictionary*:

In [ ]:
event_dict = {
    "auditory/left": 1,
    "auditory/right": 2,
    "visual/left": 3,
    "visual/right": 4,
    "smiley": 5,
    "buttonpress": 32,
}

Event dictionaries like this one are used when extracting epochs from
continuous data, and the resulting `~mne.Epochs` object allows pooling
by requesting partial trial descriptors. For example, if we wanted to pool
all auditory trials, instead of merging Event IDs 1 and 2 using the
`~mne.merge_events` function, we can make use of the fact that the keys
of ``event_dict`` contain multiple trial descriptors separated by ``/``
characters: requesting ``'auditory'`` trials will select all epochs with
Event IDs 1 and 2; requesting ``'left'`` trials will select all epochs with
Event IDs 1 and 3. An example of this is shown later, in the
`tut-section-subselect-epochs` section of the tutorial
`tut-epochs-class`.


## Plotting events

Another use of event dictionaries is when plotting events, which can serve as
a useful check that your event signals were properly sent to the STIM
channel(s) and that MNE-Python has successfully found them. The function
`mne.viz.plot_events` will plot each event versus its sample number
(or, if you provide the sampling frequency, it will plot them versus time in
seconds). It can also account for the offset between sample number and sample
index in Neuromag systems, with the ``first_samp`` parameter. If an event
dictionary is provided, it will be used to generate a legend:

In [ ]:
fig = mne.viz.plot_events(
    events, sfreq=raw.info["sfreq"], first_samp=raw.first_samp, event_id=event_dict
)

### Plotting events and raw data together

Events can also be plotted alongside the `~mne.io.Raw` object they
were extracted from, by passing the Event array as the ``events`` parameter
of `raw.plot <mne.io.Raw.plot>`:

In [ ]:
raw.plot(
    events=events,
    start=5,
    duration=10,
    color="gray",
    event_color={1: "r", 2: "g", 3: "b", 4: "m", 5: "y", 32: "k"},
)

### Making equally-spaced Events arrays

For some experiments (such as those intending to analyze resting-state
activity) there may not be any experimental events included in the raw
recording. In such cases, an Events array of equally-spaced events can be
generated using `mne.make_fixed_length_events`:

In [ ]:
new_events = mne.make_fixed_length_events(raw, start=5, stop=50, duration=2.0)

By default, the events will all be given the integer Event ID of ``1``, but
you can change that with the ``id`` parameter. It is also possible to specify
an ``overlap`` duration — i.e., if you ultimately want `epochs` that
are 2.5 seconds long, but you want them to overlap by 0.5 seconds, you can
specify ``duration=2.5, overlap=0.5`` in the call to
`~mne.make_fixed_length_events` (this will yield the same spacing of
events as ``duration=2, overlap=0)``.

## 3. Annotating continuous data

This tutorial describes adding annotations to a `~mne.io.Raw` object,
and how annotations are used in later stages of data processing.

As usual we'll start by importing the modules we need, loading some
`example data <sample-dataset>`, and (since we won't actually analyze the
raw data in this tutorial) cropping the `~mne.io.Raw` object to just 60
seconds before loading it into RAM to save memory:

In [ ]:
import os
from datetime import timedelta

import mne

sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(
    sample_data_folder, "MEG", "sample", "sample_audvis_raw.fif"
)
raw = mne.io.read_raw_fif(sample_data_raw_file, verbose=False)
raw.crop(tmax=60).load_data()

`~mne.Annotations` in MNE-Python are a way of storing short strings of
information about temporal spans of a `~mne.io.Raw` object. Below the
surface, `~mne.Annotations` are `list-like <list>` objects,
where each element comprises three pieces of information: an ``onset`` time
(in seconds), a ``duration`` (also in seconds), and a ``description`` (a text
string). Additionally, the `~mne.Annotations` object itself also keeps
track of ``orig_time``, which is a `POSIX timestamp`_ denoting a real-world
time relative to which the annotation onsets should be interpreted.



### Creating annotations programmatically

If you know in advance what spans of the `~mne.io.Raw` object you want
to annotate, `~mne.Annotations` can be created programmatically, and
you can even pass lists or arrays to the `~mne.Annotations`
constructor to annotate multiple spans at once:

In [ ]:
my_annot = mne.Annotations(
    onset=[3, 5, 7],  # in seconds
    duration=[1, 0.5, 0.25],  # in seconds, too
    description=["AAA", "BBB", "CCC"],
)
print(my_annot)

Notice that ``orig_time`` is ``None``, because we haven't specified it. In
those cases, when you add the annotations to a `~mne.io.Raw` object,
it is assumed that the ``orig_time`` matches the time of the first sample of
the recording, so ``orig_time`` will be set to match the recording
measurement date (``raw.info['meas_date']``).

In [ ]:
raw.set_annotations(my_annot)
print(raw.annotations)

# convert meas_date (a tuple of seconds, microseconds) into a float:
meas_date = raw.info["meas_date"]
orig_time = raw.annotations.orig_time
print(meas_date == orig_time)

Since the example data comes from a Neuromag system that starts counting
sample numbers before the recording begins, adding ``my_annot`` to the
`~mne.io.Raw` object also involved another automatic change: an offset
equalling the time of the first recorded sample (``raw.first_samp /
raw.info['sfreq']``) was added to the ``onset`` values of each annotation
(see `time-as-index` for more info on ``raw.first_samp``):

In [ ]:
time_of_first_sample = raw.first_samp / raw.info["sfreq"]
print(my_annot.onset + time_of_first_sample)
print(raw.annotations.onset)

If you know that your annotation onsets are relative to some other time, you
can set ``orig_time`` before you call `~mne.io.Raw.set_annotations`,
and the onset times will get adjusted based on the time difference between
your specified ``orig_time`` and ``raw.info['meas_date']``, but without the
additional adjustment for ``raw.first_samp``. ``orig_time`` can be specified
in various ways (see the documentation of `~mne.Annotations` for the
options); here we'll use an `ISO 8601`_ formatted string, and set it to be 50
seconds later than ``raw.info['meas_date']``.

In [ ]:
time_format = "%Y-%m-%d %H:%M:%S.%f"
new_orig_time = (meas_date + timedelta(seconds=50)).strftime(time_format)
print(new_orig_time)

later_annot = mne.Annotations(
    onset=[3, 5, 7],
    duration=[1, 0.5, 0.25],
    description=["DDD", "EEE", "FFF"],
    orig_time=new_orig_time,
)

raw2 = raw.copy().set_annotations(later_annot)
print(later_annot.onset)
print(raw2.annotations.onset)

<div class="alert alert-info"><h4>Note</h4><p>If your annotations fall outside the range of data times in the
    `~mne.io.Raw` object, the annotations outside the data range will
    not be added to ``raw.annotations``, and a warning will be issued.</p></div>

Now that your annotations have been added to a `~mne.io.Raw` object,
you can see them when you visualize the `~mne.io.Raw` object:

In [ ]:
fig = raw.plot(start=2, duration=6)

The three annotations appear as differently colored rectangles because they
have different ``description`` values (which are printed along the top
edge of the plot area). Notice also that colored spans appear in the small
scroll bar at the bottom of the plot window, making it easy to quickly view
where in a `~mne.io.Raw` object the annotations are so you can easily
browse through the data to find and examine them.


### Annotating Raw objects interactively

Annotations can also be added to a `~mne.io.Raw` object interactively
by clicking-and-dragging the mouse in the plot window. To do this, you must
first enter "annotation mode" by pressing `a` while the plot window is
focused; this will bring up the annotation controls:

In [ ]:
fig = raw.plot(start=2, duration=6)
fig.fake_keypress("a")

The drop-down-menu on the left determines which existing label will be
created by the next click-and-drag operation in the main plot window. New
annotation descriptions can be added by clicking the `Add
description` button; the new description will be added to the list of
descriptions and automatically selected.
The following functions relate to which description is currently selected in
the drop-down-menu:
With `Remove description` you can remove description
including the annotations.
With `Edit description` you can edit
the description of either only one annotation (the one currently selected)
or all annotations of a description.
With `Set Visible` you can show or hide descriptions.

During interactive annotation it is also possible to adjust the start and end
times of existing annotations, by clicking-and-dragging on the left or right
edges of the highlighting rectangle corresponding to that annotation. When
an annotation is selected (the background of the label at the bottom changes
to darker) the values for start and stop are visible in two spinboxes and
can also be edited there.

<div class="alert alert-danger"><h4>Warning</h4><p>Calling :meth:`~mne.io.Raw.set_annotations` **replaces** any annotations
    currently stored in the `~mne.io.Raw` object, so be careful when
    working with annotations that were created interactively (you could lose
    a lot of work if you accidentally overwrite your interactive
    annotations). A good safeguard is to run
    ``interactive_annot = raw.annotations`` after you finish an interactive
    annotation session, so that the annotations are stored in a separate
    variable outside the `~mne.io.Raw` object.</p></div>


### How annotations affect preprocessing and analysis

You may have noticed that the description for new labels in the annotation
controls window defaults to ``BAD_``. The reason for this is that annotation
is often used to mark bad temporal spans of data (such as movement artifacts
or environmental interference that cannot be removed in other ways such as
`projection <tut-projectors-background>` or filtering). Several
MNE-Python operations
are "annotation aware" and will avoid using data that is annotated with a
description that begins with "bad" or "BAD"; such operations typically have a
boolean ``reject_by_annotation`` parameter. Examples of such operations are
independent components analysis (`mne.preprocessing.ICA`), functions
for finding heartbeat and blink artifacts
(`~mne.preprocessing.find_ecg_events`,
`~mne.preprocessing.find_eog_events`), and creation of epoched data
from continuous data (`mne.Epochs`). See `tut-reject-data-spans`
for details.


### Operations on Annotations objects

`~mne.Annotations` objects can be combined by simply adding them with
the ``+`` operator, as long as they share the same ``orig_time``:

In [ ]:
new_annot = mne.Annotations(onset=3.75, duration=0.75, description="AAA")
raw.set_annotations(my_annot + new_annot)
raw.plot(start=2, duration=6)

Notice that it is possible to create overlapping annotations, even when they
share the same description. This is *not* possible when annotating
interactively; click-and-dragging to create a new annotation that overlaps
with an existing annotation with the same description will cause the old and
new annotations to be merged.

Individual annotations can be accessed by indexing an
`~mne.Annotations` object, and subsets of the annotations can be
achieved by either slicing or indexing with a list, tuple, or array of
indices:

In [ ]:
print(raw.annotations[0])  # just the first annotation
print(raw.annotations[:2])  # the first two annotations
print(raw.annotations[(3, 2)])  # the fourth and third annotations

You can also iterate over the annotations within an `~mne.Annotations`
object:

In [ ]:
for ann in raw.annotations:
    descr = ann["description"]
    start = ann["onset"]
    end = ann["onset"] + ann["duration"]
    print(f"'{descr}' goes from {start} to {end}")

Note that iterating, indexing and slicing `~mne.Annotations` all
return a copy, so changes to an indexed, sliced, or iterated element will not
modify the original `~mne.Annotations` object.

In [ ]:
# later_annot WILL be changed, because we're modifying the first element of
# later_annot.onset directly:
later_annot.onset[0] = 99

# later_annot WILL NOT be changed, because later_annot[0] returns a copy
# before the 'onset' field is changed:
later_annot[0]["onset"] = 77

print(later_annot[0]["onset"])

### Reading and writing Annotations to/from a file

`~mne.Annotations` objects have a `~mne.Annotations.save` method
which can write `.fif`, `.csv`, and `.txt` formats (the
format to write is inferred from the file extension in the filename you
provide). Be aware that the format of the onset information that is written
to the file depends on the file extension. While `.csv` files store the
onset as timestamps, `.txt` files write floats (in seconds). There is a
corresponding `~mne.read_annotations` function to load them from disk:

In [ ]:
raw.annotations.save("saved-annotations.csv", overwrite=True)
annot_from_file = mne.read_annotations("saved-annotations.csv")
print(annot_from_file)

## 4. Built-in plotting methods for Raw objects

This tutorial shows how to plot continuous data as a time series, how to plot the
spectral density of continuous data, and how to plot the sensor locations and projectors
stored in `~mne.io.Raw` objects.

As usual we'll start by importing the modules we need, loading some
`example data <sample-dataset>`, and cropping the `~mne.io.Raw` object to just 60
seconds before loading it into RAM to save memory:

In [ ]:
import os

import mne

sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(
    sample_data_folder, "MEG", "sample", "sample_audvis_raw.fif"
)
raw = mne.io.read_raw_fif(sample_data_raw_file)
raw.crop(tmax=60).load_data()

We've seen in `a previous tutorial <tut-raw-class>` how to plot data
from a `~mne.io.Raw` object using `matplotlib <matplotlib:index>`,
but `~mne.io.Raw` objects also have several built-in plotting methods:

- `~mne.io.Raw.plot`
- `~mne.io.Raw.plot_sensors`
- `~mne.io.Raw.plot_projs_topomap`

The first one is discussed here in detail; the last two are shown briefly
and covered in-depth in other tutorials. This tutorial also covers a few
ways of plotting the spectral content of :class:`~mne.io.Raw` data.


### Interactive data browsing with ``Raw.plot()``

The `~mne.io.Raw.plot` method of `~mne.io.Raw` objects provides
a versatile interface for exploring continuous data. For interactive viewing
and data quality checking, it can be called with no additional parameters:

In [ ]:
raw.plot()

It may not be obvious when viewing this tutorial online, but by default, the
`~mne.io.Raw.plot` method generates an *interactive* plot window with
several useful features:

- It spaces the channels equally along the y-axis.

  - 20 channels are shown by default; you can scroll through the channels
    using the `↑` and `↓` arrow keys, or by clicking on the
    colored scroll bar on the right edge of the plot.

  - The number of visible channels can be adjusted by the ``n_channels``
    parameter, or changed interactively using `page up` and |`page
    down` keys.

  - You can toggle the display to "butterfly" mode (superimposing all
    channels of the same type on top of one another) by pressing `b`,
    or start in butterfly mode by passing the ``butterfly=True`` parameter.

- It shows the first 10 seconds of the `~mne.io.Raw` object.

  - You can shorten or lengthen the window length using `home` and
    `end` keys, or start with a specific window duration by passing the
    ``duration`` parameter.

  - You can scroll in the time domain using the `←` and
    `→` arrow keys, or start at a specific point by passing the
    ``start`` parameter. Scrolling using `shift``→` or
    `shift``←` scrolls a full window width at a time.

- It allows clicking on channels to mark/unmark as "bad".

  - When the plot window is closed, the `~mne.io.Raw` object's
    ``info`` attribute will be updated, adding or removing the newly
    (un)marked channels to/from the `~mne.Info` object's ``bads``
    field (A.K.A. ``raw.info['bads']``).

- It allows interactive `annotation <annotations>` of the raw data.

  - This allows you to mark time spans that should be excluded from future
    computations due to large movement artifacts, line noise, or other
    distortions of the signal. Annotation mode is entered by pressing
    :kbd:`a`. See `annotations-tutorial` for details.

- It automatically applies any `projectors <projector>` before plotting
  the data.

  - These can be enabled/disabled interactively by clicking the ``Proj``
    button at the lower right corner of the plot window, or disabled by
    default by passing the ``proj=False`` parameter. See
    `tut-projectors-background` for more info on projectors.

These and other keyboard shortcuts are listed in the Help window, accessed
through the ``Help`` button at the lower left corner of the plot window.
Other plot properties (such as color of the channel traces, channel order and
grouping, simultaneous plotting of `events`, scaling, clipping,
filtering, etc.) can also be adjusted through parameters passed to the
`~mne.io.Raw.plot` method; see the docstring for details.


## Plotting spectral density of continuous data

To visualize the frequency content of continuous data, the `~mne.io.Raw`
object provides a `~mne.io.Raw.compute_psd` method to compute
`spectral density`_ and the resulting `~mne.time_frequency.Spectrum`
object has a `~mne.time_frequency.Spectrum.plot` method:

In [ ]:
spectrum = raw.compute_psd()
spectrum.plot(average=True, picks="data", exclude="bads", amplitude=False)

If the data have been filtered, vertical dashed lines will automatically
indicate filter boundaries. The spectrum for each channel type is drawn in
its own subplot; here we've passed the ``average=True`` parameter to get a
summary for each channel type, but it is also possible to plot each channel
individually, with options for how the spectrum should be computed,
color-coding the channels by location, and more. For example, here is a plot
of just a few sensors (specified with the ``picks`` parameter), color-coded
by spatial location (via the ``spatial_colors`` parameter, see the
documentation of `~mne.time_frequency.Spectrum.plot` for full details):

In [ ]:
midline = ["EEG 002", "EEG 012", "EEG 030", "EEG 048", "EEG 058", "EEG 060"]
spectrum.plot(picks=midline, exclude="bads", amplitude=False)

It is also possible to plot spectral power estimates across sensors as a
scalp topography, using the `~mne.time_frequency.Spectrum`'s
`~mne.time_frequency.Spectrum.plot_topomap` method. The default
parameters will plot five frequency bands (δ, θ, α, β, γ), will compute power
based on magnetometer channels (if present), and will plot the power
estimates on a dB-like log-scale:

In [ ]:
spectrum.plot_topomap()

Alternatively, you can plot the PSD for every sensor on its own axes, with
the axes arranged spatially to correspond to sensor locations in space, using
`~mne.time_frequency.Spectrum.plot_topo`:

In [ ]:
spectrum.plot_topo()

This plot is also interactive; hovering over each "thumbnail" plot will
display the channel name in the bottom left of the plot window, and clicking
on a thumbnail plot will create a second figure showing a larger version of
the selected channel's spectral density (as if you had called
`~mne.time_frequency.Spectrum.plot` with that channel passed as ``picks``).

By default, `~mne.time_frequency.Spectrum.plot_topo` will show only the MEG
channels if MEG channels are present; if only EEG channels are found, they
will be plotted instead:

In [ ]:
spectrum.pick("eeg").plot_topo()

<div class="alert alert-info"><h4>Note</h4><p>Prior to the addition of the :class:`~mne.time_frequency.Spectrum` class,
   the above plots were possible via::

       raw.plot_psd(average=True)
       raw.plot_psd_topo()
       raw.pick('eeg').plot_psd_topo()

   (there was no ``plot_topomap`` method for `~mne.io.Raw`). The
   `~mne.io.Raw.plot_psd` and `~mne.io.Raw.plot_psd_topo` methods
   of `~mne.io.Raw` objects are still provided to support legacy
   analysis scripts, but new code should instead use the
   `~mne.time_frequency.Spectrum` object API.</p></div>


### Plotting sensor locations from ``Raw`` objects

The channel locations in a `~mne.io.Raw` object can be easily plotted
with the `~mne.io.Raw.plot_sensors` method. A brief example is shown
here; notice that channels in ``raw.info['bads']`` are plotted in red. More
details and additional examples are given in the tutorial
`tut-sensor-locations`.

In [ ]:
raw.plot_sensors(ch_type="eeg")

### Plotting projectors from ``Raw`` objects

As seen in the output of `mne.io.read_raw_fif` above, there are
`projectors <projector>` included in the example `~mne.io.Raw`
file (representing environmental noise in the signal, so it can later be
"projected out" during preprocessing). You can visualize these projectors
using the `~mne.io.Raw.plot_projs_topomap` method. By default it will
show one figure per channel type for which projectors are present, and each
figure will have one subplot per projector. The three projectors in this file
were only computed for magnetometers, so one figure with three subplots is
generated. More details on working with and plotting projectors are given in
`tut-projectors-background` and `tut-artifact-ssp`.

In [ ]:
raw.plot_projs_topomap(colorbar=True)